In [182]:
year = 2023
day = 14

In [183]:
from aocd import submit
from aocd.models import Puzzle
import numpy as np

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
# data = puzzle.examples[0].input_data

data = data.strip()
data = data.split("\n")

data = [list(line) for line in data]
data = np.array(data)
data

array([['.', '.', '#', ..., '#', '.', '.'],
       ['.', 'O', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., 'O', '.', '.'],
       ...,
       ['O', '.', '.', ..., '.', 'O', 'O'],
       ['.', '#', '.', ..., '.', 'O', '.'],
       ['.', 'O', '.', ..., '.', 'O', '.']], dtype='<U1')

In [192]:
def tilt(data, direction):
    while True:
        if direction == "N":
            mask = (data[:-1] == ".") & (data[1:, :] == "O")
            move_to = np.vstack((mask, np.full((1,data.shape[1]), False)))
            move_from = np.vstack((np.full((1,data.shape[1]), False), mask))
        elif direction == "S":
            mask = (data[1:] == ".") & (data[:-1, :] == "O")
            move_to = np.vstack((np.full((1,data.shape[1]), False), mask))
            move_from = np.vstack((mask, np.full((1,data.shape[1]), False)))
        elif direction == "E":
            mask = (data[:, 1:] == ".") & (data[:, :-1] == "O")
            move_to = np.hstack((np.full((data.shape[0], 1), False), mask))
            move_from = np.hstack((mask, np.full((data.shape[0], 1), False)))
        elif direction == "W":
            mask = (data[:, :-1] == ".") & (data[:, 1:] == "O")
            move_to = np.hstack((mask, np.full((data.shape[0], 1), False)))
            move_from = np.hstack((np.full((data.shape[0], 1), False), mask))
        else:
            raise ValueError(f"Unknown direction {direction}")
        if not np.any(mask):
            break
        data[move_from] = "."
        data[move_to] = "O"

    return data

In [193]:
def calculate_score(data):
    H, W = data.shape
    total = 0
    for x in range(W):
        for y in range(H):
            if data[y, x] == "O":
                total += H-y
    return total

In [194]:
answer = calculate_score(tilt(data.copy(), "N"))
submit(answer, part="a", year=year, day=day)

aocd will not submit that answer again. At 2023-12-14 00:12:34.190351-05:00 you've previously submitted 113424 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [195]:
def cycle(data):
    data = tilt(data, "N")
    data = tilt(data, "W")
    data = tilt(data, "S")
    data = tilt(data, "E")
    return data

In [196]:
def find_repeats(seq):
    # start from 2 to fix issue with repeating score but not repeating board
    for i in range(2, len(seq) // 2):
        if seq[-i:] == seq[-2*i:-i]:
            return i
    return None

In [197]:
# MAX_TRIES is a heuristic to prevent it looping too long
MAX_TRIES = 200
def cycle_until_repeats(data):
    scores = []
    for i in range(MAX_TRIES):
        s = calculate_score(data)
        print(i, s)
        data = cycle(data)
        # bit of an assumption here:
        # we want to detect repeating patterns in the board positions.
        # we do this by detection repeating patterns in board score,
        # but board score is not a unique hash of board position.
        if cyc := find_repeats(scores):
            return cyc, scores
        scores.append(s)
    return 0, scores


In [198]:
cycle_len, scores = cycle_until_repeats(data.copy())

0 105314
1 104345
2 104167
3 103875
4 103585
5 103442
6 103254
7 103071
8 102834
9 102569
10 102275
11 102022
12 101737
13 101517
14 101336
15 101159
16 100999
17 100857
18 100681
19 100541
20 100341
21 100177
22 100002
23 99835
24 99657
25 99495
26 99299
27 99138
28 98971
29 98821
30 98651
31 98491
32 98291
33 98128
34 97958
35 97819
36 97662
37 97540
38 97391
39 97267
40 97129
41 97037
42 96934
43 96858
44 96773
45 96736
46 96663
47 96626
48 96573
49 96580
50 96583
51 96622
52 96647
53 96681
54 96704
55 96760
56 96807
57 96864
58 96888
59 96899
60 96898
61 96859
62 96813
63 96776
64 96765
65 96770
66 96849
67 96963
68 97051
69 97150
70 97218
71 97277
72 97277
73 97257
74 97202
75 97170
76 97161
77 97153
78 97149
79 97156
80 97137
81 97121
82 97072
83 97042
84 96986
85 96921
86 96852
87 96780
88 96714
89 96643
90 96566
91 96500
92 96432
93 96359
94 96276
95 96215
96 96174
97 96121
98 96079
99 96044
100 96014
101 95990
102 95972
103 95962
104 95961
105 95981
106 96001
107 96020
108 960

In [199]:

offset = len(scores) % cycle_len
answer_idx = 1_000_000_000 % cycle_len
answer = scores[-cycle_len+(answer_idx-offset)%cycle_len]


96003